In [ ]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import os
import datetime


file_path = "./data/"
folder_names = [f for f in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, f))]
agency_df = None
stops_df = None
routes_df = None
trips_df = None
stop_times_df = None

for folder_name in folder_names:
    try:
        #agency
        new_agency_df = pd.read_csv(file_path + folder_name + "/agency.txt")
        new_agency_df["date"] = folder_name
        agency_df = pd.concat([agency_df, new_agency_df], ignore_index=True)

        #stops
        new_stops_df = pd.read_csv(file_path + folder_name + "/stops.txt")
        new_stops_df["date"] = folder_name
        stops_df = pd.concat([stops_df, new_stops_df], ignore_index=True)

        #routes
        new_routes_df = pd.read_csv(file_path + folder_name + "/routes.txt")
        new_routes_df["date"] = folder_name
        routes_df = pd.concat([routes_df, new_routes_df], ignore_index=True)

        #trips
        new_trips_df = pd.read_csv(file_path + folder_name + "/trips.txt")
        new_trips_df["date"] = folder_name
        trips_df = pd.concat([trips_df, new_trips_df], ignore_index=True)

        #stop_times
        new_stop_times_df = pd.read_csv(file_path + folder_name + "/stop_times.txt")
        new_stop_times_df["date"] = folder_name
        stop_times_df = pd.concat([stop_times_df, new_stop_times_df], ignore_index=True)
        
    except ValueError:
        print("Error")




# Crea il widget DataFrame
df_widget = widgets.Output()
with df_widget:
    display(stops_df)

# Visualizza il widget DataFrame
display(df_widget)


In [ ]:
#Pura mappa
fig = px.scatter_mapbox(stops_df, 
                        lat="stop_lat", lon="stop_lon",
                        hover_name="stop_name", hover_data=["stop_id"],
                        zoom=4, height=700)
fig.update_layout(mapbox_style="open-street-map")

In [ ]:
fig = px.scatter_mapbox(stops_df, 
                        lat="stop_lat", lon="stop_lon",
                        hover_name="stop_name", hover_data=["stop_id"],
                        zoom=4, height=700)
#fig.update_layout(mapbox_style="open-street-map")
# Aggiunge uno slider per filtrare i dati
slider = widgets.IntRangeSlider(value=[2017, 2021], min=2017, max=2021, step=1,
                                description="Anno", layout=widgets.Layout(width="80%"))
output = widgets.Output()

def update_map(change):
    # Filtra i dati in base al valore dello slider
    # Converti la colonna "data" in un tipo datetime
    stops_df["date"] = pd.to_datetime(stops_df["date"])

    # Filtra le righe in base alla data
    filtered_df = stops_df.loc[(stops_df["date"] >= datetime.datetime(change.new[0], 1, 1)) & (stops_df["date"] <= datetime.datetime(change.new[1], 12, 31))]
   
    
    # Aggiorna la mappa Mapbox con i dati filtrati
    fig.data[0].lat = filtered_df["stop_lat"]
    fig.data[0].lon = filtered_df["stop_lon"]
    #fig.data[0].customdata = filtered_df[["nome", "tipo", "indirizzo", "città"]]

slider.observe(update_map, names="value")
display(widgets.VBox([slider, output]))

# Visualizza la mappa Mapbox
fig.show()

In [ ]:
from ipywidgets import Output
from IPython.display import clear_output


stops_df['date'] = pd.to_datetime(stops_df['date'], format='%Y%m%d')

# Creazione dello slider per selezionare l'anno
year_slider = widgets.SelectionSlider(
    options=[2017, 2018, 2019, 2020, 2021],
    value=2017,
    description='Seleziona l\'anno:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

# Funzione per filtrare i dati in base all'anno selezionato
def update_figure(selected_year):
    filtered_df = stops_df[stops_df['date'].dt.year == selected_year]
    fig = px.scatter_mapbox(filtered_df,
                            lat="stop_lat", lon="stop_lon",
                            hover_name="stop_name", hover_data=["stop_id"],
                            zoom=4, height=700)
    with fig_output:
        clear_output(wait=True)
        display(fig)

# Inserimento dello slider
display(year_slider)

# Creazione della mappa Mapbox iniziale
fig = px.scatter_mapbox(stops_df,
                        lat="stop_lat", lon="stop_lon",
                        hover_name="stop_name", hover_data=["stop_id"],
                        zoom=4, height=700)

# Visualizzazione della mappa Mapbox
fig_output = Output()
display(fig_output)
with fig_output:
    clear_output(wait=True)

# Collegamento dello slider alla funzione di aggiornamento della mappa
widgets.interact(update_figure, selected_year=year_slider);

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

# Crea la mappa iniziale utilizzando Plotly Express
fig = px.scatter_mapbox(stops_df,
                        lat="stop_lat", lon="stop_lon",
                        hover_name="stop_name", hover_data=["stop_id"],
                        zoom=4, height=700)

# Crea lo slider per selezionare l'anno
year_slider = widgets.IntRangeSlider(
    value=[2017, 2021],
    min=2017,
    max=2021,
    step=1,
    description='Anno:',
    continuous_update=False,
    orientation='horizontal'
)

# Definisci la funzione di aggiornamento della mappa
def update_map(year_range):
    filtered_data = stops_df[(stops_df['date'] >= year_range[0]) & (stops_df['date'] <= year_range[1])]
    fig.data[0].lat = filtered_data['stop_lat']
    fig.data[0].lon = filtered_data['stop_lon']
    fig.data[0].hovertext = filtered_data['stop_name']
    fig.data[0].hovertemplate = "<b>%{hovertext}</b><br>Stop ID: %{customdata[0]}"

    fig.update_layout(transition_duration=500)

    fig.show()

# Collega lo slider alla funzione di aggiornamento
widgets.interact(update_map, year_range=year_slider)

# Mostra la mappa iniziale
fig.show()